In [7]:
import pandas as pd
import numpy as np

data_df = pd.read_csv("c3_bike-sharing.csv")

In [4]:
temp = data_df.temp.values
users = data_df.users.values

data_df.head()

,temp,users
0,0.1964,120
1,0.2000,108
2,0.2270,82
3,0.2043,88
4,0.1508,41


In [5]:
# collinearity appears when there is an exact linear relationship between one of more of the features

# test by converting the temp to celsius

temp_C = 47 * temp - 8

In [13]:
# illustrate OLS solution does not exist

# create input matrix
X = np.c_[temp, temp_C]

# add ones
X1 = np.c_[np.ones(X.shape[0]), X]

# compute rank
rank = np.linalg.matrix_rank(X1)
print("Rank:", rank)

Rank: 2


Rank deficient as rank is not maximal ie 3

In [14]:
from scipy.linalg import lstsq

# compute OLS using lstsq
w, rss, rank, _ = lstsq(X1, users)

print("w:", w)
print("rank:", rank)
print("RSS:", rss)

w: [155.34445517  27.10638524  31.24446504]
rank: 2
RSS: []


In [17]:
# compare performance

from sklearn.metrics import r2_score

# R^2 coeff of simple linear regresiion
coefs = np.polyfit(temp, users, deg=1)
y_pred_normal = np.polyval(coefs, temp)
r2_normal = r2_score(users, y_pred_normal)
print("R^2 normal:", r2_normal)

R^2 normal: 0.5954233080185317


In [18]:
# r^2 coefficient with collinear features
y_pred_collinear = np.matmul(X1, w)
r2_collinear = r2_score(users, y_pred_collinear)
print("R^2 collinear:", r2_collinear)

R^2 collinear: 0.5954233080185317


Testing nearly collinear features

In [19]:
# convert to degrees celsius to fahrenheit
temp_F = 1.8 * temp_C + 32

In [22]:
# add small noise to temp_F variable

def compute_ols_with_noise(temp_c, users):
    # convert to deg F
    temp_F = 1.8 * temp_C + 32
    
    # add small variations
    noise = np.random.normal(loc=0, scale=0.01, size=temp_F.shape)
    temp_F += noise
    
    # create input matrix X
    X = np.c_[temp_C, temp_F]
    
    # compute OLS using lstsq
    X1 = np.c_[np.ones(X.shape[0]), X]
    w, rss, rank, _ = lstsq(X1, users)
    
    return w, rss, rank, X1


w, rss, rank, X1 = compute_ols_with_noise(temp_C, users)

print("rank:", rank)
print("RMSE:", np.sqrt(rss / len(users)))
print("w:", w)

rank: 3
RMSE: 232.95329660235728
w: [-42213.79222862  -2351.85666849   1324.23221356]


In [26]:
# run several times see coefficients vary a lot

txt_fmt = "{:<5}{:<6}{:<20}{:}"
print(txt_fmt.format("run", "rank", "RMSE", "coefficients"))

for i in range(5):
    w, rss, rank, X1 = compute_ols_with_noise(temp_C, users)
    print(txt_fmt.format(i, rank, np.sqrt(rss/ len(users)), w))

run  rank  RMSE                coefficients
0    3     232.47004192329436  [64400.06978346  3645.62699786 -2007.59786619]
1    3     233.35151569103408  [9268.50425309  544.16315104 -284.63940954]
2    3     233.19960790857675  [-29072.27514584  -1612.36352665    913.46536364]
3    3     233.2570038885482   [-24689.63064063  -1366.07997074    776.5800606 ]
4    3     232.7793255309521   [-53089.39037663  -2963.37666439   1664.0040854 ]


ill-conditioning

In [28]:
# inverting a matrix with large condition number is numerically unstable

# condition number
cn = np.linalg.cond(X1)
print("Condition number:", cn)

Condition number: 213799.11622193834


In [30]:
# same with nearly collinear matrix
y_pred_nearcol = np.matmul(X1, w)
r2_nearcol = r2_score(users, y_pred_nearcol)

print("R^2 nearly collinear:", r2_nearcol)

R^2 nearly collinear: 0.597458040904572


In [32]:
# solve ill conditioning using constraint on coefficients. Add regularization term that penalizes large coefficients

from sklearn.linear_model import Ridge

def compute_with_regularization(temp_C, users):
    
     # add small variations
    noise = np.random.normal(loc=0, scale=0.01, size=temp_C.shape)
    temp_F = (1.8 * temp_C + 32) + noise
    
    # create input matrix X
    X = np.c_[temp_C, temp_F]
    
    # fit ridge regression
    ridge = Ridge(alpha=100)
    ridge.fit(X, users)
    
    return ridge, X

ridge, X = compute_with_regularization(temp_C, users)

print("Coefficients:", ridge.coef_)
print("Intercept:", ridge.intercept_)
print("R^2:", ridge.score(X, users))
    
    

Coefficients: [ 7.39498492 13.55373162]
Intercept: -273.2687438185834
R^2: 0.5954391250709111


In [33]:
txt_fmt = "{:<5}{:<27}{:<21}{:}"
print(txt_fmt.format("run", "coefficients", "intercept", "R^2"))
for i in range(5):
    ridge, X = compute_with_regularization(temp_C, users)
    print(txt_fmt.format(i, str(list(ridge.coef_.round(8))),
                         ridge.intercept_, ridge.score(X, users)))

run  coefficients               intercept            R^2
0    [7.55336854, 13.46406903]  -270.355577608103    0.5954140728932338
1    [7.26542626, 13.62463543]  -275.50596646476356  0.5954595200772208
2    [7.55898202, 13.46119364]  -270.27511547311985  0.5954132298144229
3    [7.27748969, 13.6179956]   -275.30058971359153  0.5954575998217178
4    [7.590474, 13.4436044]     -269.71237091509147  0.5954083138188453
